 ## Create the data

In [1]:
# import required modules
from mimesis.locales import Locale
from mimesis.keys import maybe
from mimesis.schema import Field, Schema
from mimesis.enums import TimestampFormat
from mimesis import Datetime

dt = Datetime()
f = Field(locale=Locale.EN_GB, seed=42)


# Add table definitions

table_definition = {
    "person":{
        "amount":1000 *10
    },
    "product":{
        "amount":1000 *10
    },
    "order":{
        "amount":10000 *10
    },
    "artist":{
        "amount":500 *10
    },
    "review":{
        "amount":2000 *10
    }
}

# Create table data

## person

def person_generator() -> dict:
    first_name = f('first_name')
    last_name = f('last_name')
    return {
        "id":"person:"+f"⟨{f('uuid')}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.9)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

person_schema = Schema(
    schema=person_generator,
    iterations=table_definition['person']['amount']
)
person_data = person_schema.create()

person_id_count = table_definition['person']['amount']-1

print("person data created")

## artist

def artist_generator() -> dict:
    first_name = f("first_name")
    last_name = f("last_name")
    return {
        "id":"artist:"+f"⟨{f('uuid')}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.5)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }


artist_schema = Schema(
    schema=artist_generator,
    iterations=table_definition['artist']['amount']
)

artist_data = artist_schema.create()

artist_id_count = table_definition['artist']['amount']-1

print("artist data created")

## product

def product_generator() -> dict:
    created_at = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)
    quantity = f('integer_number', start=0, end=20)
    return {
        "id":"product:"+f"⟨{f('uuid')}⟩",
        "name":' '.join(f('words', quantity=2)),
        "description":' '.join(f('words', quantity=f('integer_number', start=8, end=25))),
        "category":f('choice', items=["oil paint", "watercolor", "acrylic paint", "charcoal", "pencil", "ink", "pastel", "collage", "digital art", "mixed media"]),
        "price":f('price', minimum=500, maximum=25000),
        "currency":f('currency_symbol'),
        "discount":f('float_number', start=0.2, end=0.8, precision=1, key=maybe(None, probability=0.8)),
        "quantity":f('integer_number', start=0, end=20), 
        "image_url":f('stock_image_url'),
        "artist":artist_data[f('integer_number', start=0, end=artist_id_count)]['id'],
        "creation_history": {
            "created_at":created_at,
            "quantity":quantity
        }
    }

product_schema = Schema(
    schema=product_generator,
    iterations=table_definition['product']['amount']
)

product_data = product_schema.create()

product_id_count = table_definition['product']['amount']-1

print("product data created")

## order

def order_generator() -> dict:
    person_number = f('integer_number', start=0, end=person_id_count)
    product_number = f('integer_number', start=0, end=product_id_count)
    shipping_address = person_data[person_number]['address']
    order_date = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)
    return {
        "id":f"order:"+f"⟨{f('uuid')}⟩",
        "in":person_data[person_number]['id'],
        "out":product_data[product_number]['id'],
        "product_name":product_data[product_number]['name'],
        "currency":product_data[product_number]['currency'],
        "discount":product_data[product_number]['discount'],
        "price":product_data[product_number]['price'],
        "quantity":f('integer_number', start=1, end=3),
        "order_date":order_date,
        "shipping_address":shipping_address,
        "payment_method":f('choice', items=['credit card','debit card', 'PayPal']),
        "order_status":f('choice', items=['pending','processing', 'shipped', 'delivered'], key=maybe(None, probability=0.1))
    }

order_schema = Schema(
    schema=order_generator,
    iterations=table_definition['order']['amount']
)

order_data = order_schema.create()

order_id_count = table_definition['order']['amount']-1

print("order data created")

## review

def review_generator() -> dict:
    return {
        "id":"review:"+f"⟨{f('uuid')}⟩",
        "person":person_data[f('integer_number', start=0, end=person_id_count)]['id'],
        "product":product_data[f('integer_number', start=0, end=product_id_count)]['id'],
        "artist":artist_data[f('integer_number', start=0, end=artist_id_count)]['id'],
        "rating":f('choice', items=[1,2,3,4,5]),
        "review_text":' '.join(f('words', quantity=f('integer_number', start=8, end=50)))
    }

review_schema = Schema(
    schema=review_generator,
    iterations=table_definition['review']['amount']
)

review_data = review_schema.create()

review_id_count = table_definition['review']['amount']-1

print("review data created")

person data created
artist data created
product data created
order data created
review data created


 ## Load the data

In [2]:
from surrealdb import SurrealDB
from uuid import uuid4

db = SurrealDB("ws://localhost:8000/test/test")

db.signin({
    "username": "root",
    "password": "root",
})


def insert_relate_statement(table_data:list[dict]) -> str:
    """
    Inserting data through relate statement
    """

    table_record_id = -1
    for record in table_data:
        table_record_id += 1
        db.query(
    f"RELATE {table_data[table_record_id]['in']} -> {table_data[table_record_id]['id']} -> {table_data[table_record_id]['out']} CONTENT {record};"
            )

In [3]:
%%time
db.query(f"INSERT INTO person {person_data}")

CPU times: user 33.1 s, sys: 659 ms, total: 33.8 s
Wall time: 34.2 s


[{'address': {'address_line_1': '1310 Blaney',
   'address_line_2': 'apt. 10',
   'city': 'Ballymena',
   'coordinates': [-84.279118, -146.269714],
   'country': 'England',
   'post_code': 'TG1C 7OP'},
  'email': 'holds1871@live.com',
  'first_name': 'Anthony',
  'id': 'person:⟨d065b779-7ae6-4f1a-9f22-be70553187aa⟩',
  'last_name': 'Reilly',
  'name': 'Anthony Reilly',
  'phone': '01500 265774'},
 {'address': {'address_line_1': '477 Glenlevan',
   'city': 'Banbridge',
   'coordinates': [46.585326, -122.522646],
   'country': 'Scotland',
   'post_code': 'OS7K 4HW'},
  'email': 'dose1974@duck.com',
  'first_name': 'Laila',
  'id': 'person:⟨262b67ef-4a7f-4627-b513-493ca48549e3⟩',
  'last_name': 'Sears',
  'name': 'Laila Sears',
  'phone': '0112 713 3320'},
 {'address': {'address_line_1': '866 Derryview',
   'city': 'Luton',
   'coordinates': [55.283089, 82.703443],
   'country': 'Scotland',
   'post_code': 'EY3C 0WP'},
  'email': 'cargo1950@protonmail.com',
  'first_name': 'Vivien',
  'id

In [4]:
%%time
db.query(f"INSERT INTO product {product_data}")

CPU times: user 11.4 s, sys: 305 ms, total: 11.7 s
Wall time: 12.2 s


[{'artist': 'artist:⟨fa2364f7-a9e0-4107-97fc-42a34e048dc9⟩',
  'category': 'pencil',
  'creation_history': {'created_at': '2023-12-05T17:21:41.832506',
   'quantity': 20},
  'currency': '£',
  'description': 'dr curve recognition please temple becomes hiv alliance crm leading alabama',
  'id': 'product:⟨0402c2f6-4bcf-49ad-8414-8efef36170bf⟩',
  'image_url': 'https://source.unsplash.com/1920x1080?',
  'name': 'page aims',
  'price': 4846.57,
  'quantity': 8},
 {'artist': 'artist:⟨6c45d630-8219-43ec-8ed8-3b14ecc2545f⟩',
  'category': 'acrylic paint',
  'creation_history': {'created_at': '2023-05-02T23:53:58.953459',
   'quantity': 7},
  'currency': '£',
  'description': 'marshall crystal mystery sh achieve served principle fear choice vintage famous bestsellers',
  'id': 'product:⟨65580ac3-4ff5-4883-8dc5-84e3b029296c⟩',
  'image_url': 'https://source.unsplash.com/1920x1080?',
  'name': 'continental past',
  'price': 11043.84,
  'quantity': 17},
 {'artist': 'artist:⟨2913454b-e226-410a-b52

In [5]:
%%time
insert_relate_statement(order_data)

CPU times: user 4min 14s, sys: 14.4 s, total: 4min 29s
Wall time: 7min 4s


In [6]:
%%time
db.query(f"INSERT INTO artist {artist_data}")

CPU times: user 16.3 s, sys: 339 ms, total: 16.6 s
Wall time: 16.9 s


[{'address': {'address_line_1': '1188 Bovennett',
   'city': 'Droylsden',
   'coordinates': [-63.222972, -59.890952],
   'country': 'Northern Ireland',
   'post_code': 'UB0E 0VX'},
  'email': 'exclusively1914@gmail.com',
  'first_name': 'Stephane',
  'id': 'artist:⟨c921d27b-1677-432f-b259-5db7d6c92b52⟩',
  'last_name': 'Wood',
  'name': 'Stephane Wood',
  'phone': '0306 298 8182'},
 {'address': {'address_line_1': '1082 Geneva',
   'city': 'Luton',
   'coordinates': [18.531479, 65.833724],
   'country': 'England',
   'post_code': 'LG3A 9QV'},
  'company_name': 'Gkn (GKN)',
  'email': 'organisation2067@example.com',
  'first_name': 'Rick',
  'id': 'artist:⟨9d7c1029-b980-4839-9cfe-7171bdb3d594⟩',
  'last_name': 'Richard',
  'name': 'Rick Richard',
  'phone': '0800 550433'},
 {'address': {'address_line_1': '1225 Bridewell',
   'address_line_2': 'Suite. 23',
   'city': 'Limavady',
   'coordinates': [-52.339201, 102.074566],
   'country': 'Northern Ireland',
   'post_code': 'QW1K 5NP'},
  'c

In [7]:
%%time
db.query(f"INSERT INTO review {review_data}")

CPU times: user 8.64 s, sys: 241 ms, total: 8.88 s
Wall time: 9.48 s


[{'artist': 'artist:⟨3cad61c1-694a-4e2a-90ed-8b5dbef8b758⟩',
  'id': 'review:⟨16c695ec-10b6-4eef-b24e-94ee4d62f835⟩',
  'person': 'person:⟨1330cadc-da5e-418f-80d0-b82848bce851⟩',
  'product': 'product:⟨b3a89cd4-e921-4e62-b456-2323bdc52c6a⟩',
  'rating': 1,
  'review_text': 'species individual nothing folder ordering tvs skype talking ultra purpose magazine mp fred sufficient milfhunter computer green conclusion stack below accommodations stretch navy concentration released electronics shown purple failure smoking different few excellence professional creator hurricane clinic genetics struggle recreation prepared mm transportation cultures columns trading companies manager'},
 {'artist': 'artist:⟨d6624db4-087c-4bd4-998d-c96740cf02cd⟩',
  'id': 'review:⟨72891873-dcc8-4663-9230-bbc4a4adf833⟩',
  'person': 'person:⟨88208766-2061-43dd-81d8-9811bf032356⟩',
  'product': 'product:⟨e2baff7b-eede-454c-89f7-94c68f819637⟩',
  'rating': 1,
  'review_text': 'manual grew results xx interaction bush h

 ## Run the queries

In [8]:
from random import randint
# getting just an array of ids to use for loops in some queries
person_ids = db.query(f"SELECT VALUE id FROM person")
product_ids = db.query(f"SELECT VALUE id FROM product")
order_ids = db.query(f"SELECT VALUE id FROM order")
artist_ids = db.query(f"SELECT VALUE id FROM artist")
review_ids = db.query(f"SELECT VALUE id FROM review")

 ### Q1: lookup vs record links

In [9]:
%%timeit
list(db.query(""" 
SELECT
    id,
    rating,
    review_text,
	artist.name,
	artist.email,
	artist.phone,
	person.name,
	person.email,
	person.phone,
	product.name,
	product.category,
	product.price
FROM review;
"""))

2.16 s ± 30 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


 ### Q2: lookup vs graph (and link)

In [10]:
%%timeit
db.query("""
SELECT
	price,
	order_date,
	product_name,
	->product.category,
	->product.description,
	->product.image_url,
	<-person.name,
	<-person.email,
	<-person.phone,
	->product.artist.name,
	->product.artist.email,
	->product.artist.phone
FROM order;
""")

The slowest run took 40.91 times longer than the fastest. This could mean that an intermediate result is being cached.
4.11 ms ± 8.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


 ### Q2: lookup vs graph - using in/out instead of arrow


In [11]:
%%timeit
db.query(""" 
SELECT
	price,
	order_date,
	product_name,
	out.category,
	out.description,
	out.image_url,
	in.person.name,
	in.person.email,
	in.person.phone,
	out.artist.name,
	out.artist.email,
	out.artist.phone
FROM order;
""")

137 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


 ### Q3: Name and email for all customers in England

In [12]:
%%time
db.query(""" 
DEFINE INDEX person_country ON TABLE person COLUMNS address.country;
""")

CPU times: user 27 µs, sys: 14 µs, total: 41 µs
Wall time: 98 µs


In [13]:
%%timeit
db.query(""" 
SELECT name, email 
FROM person 
WHERE address.country = "England";	
""")

78.7 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


 ### Q4: Count the number of confirmed orders in Q1 by artists in England

In [14]:
# TODO fix index
%%time
db.query(""" 
DEFINE INDEX order_count ON TABLE order COLUMNS order_status, order_date, address.country;
""")

UsageError: Line magic function `%%time` not found.


In [15]:
%%timeit
db.query(""" 
SELECT count() FROM order
WHERE (order_status != "pending"
OR order_status = null)
AND time::month(<datetime>order_date) <=3
AND ->product.artist.address.country ?= "England"
GROUP ALL;
""")


126 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


 ### Q5: Delete a specific review

In [16]:
%%time
db.query(f""" 
DELETE {review_ids[0]};
""")


CPU times: user 30 µs, sys: 15 µs, total: 45 µs
Wall time: 102 µs


 ### Q6: Delete reviews from a particular category

In [17]:
%%time
db.query(""" 
DELETE review
WHERE product.category = "charcoal";
""")

CPU times: user 40 µs, sys: 11 µs, total: 51 µs
Wall time: 105 µs


 ### Q7: Update a customer address

In [18]:
%%timeit
db.query(f"UPDATE {person_ids[randint(0, person_id_count)]}"+"""
SET address = {
	'address_line_1': '497 Ballycander',
	'address_line_2': None,
	'city': 'Bromyard',
	'country': 'Wales',
	'post_code': 'ZX8N 4VJ',
	'coordinates': [68.772592, -35.491877]
	}
RETURN NONE;
""")

332 µs ± 45.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


 ### Q8: Update discounts for products

In [19]:
%%time
db.query(""" 
UPDATE product
SET discount = 0.2
WHERE price < 1000
RETURN NONE;
""")


CPU times: user 39 µs, sys: 17 µs, total: 56 µs
Wall time: 198 µs


 ### Q9: Transaction - order from a new customer

In [20]:
%%timeit
random_person_id = person_ids[randint(0, person_id_count)]
random_product_id = product_ids[randint(0, product_id_count)]

db.query(""" 
# Transaction - order from a new customer
BEGIN TRANSACTION;
-- insert into the person table
CREATE person CONTENT {
	"""+f"'id': {random_person_id},"+"""
	'first_name': 'Karyl',
	'last_name': 'Langley',
	'name': 'Karyl Langley',
	'company_name': None,
	'email': 'dee1961@gmail.com',
	'phone': '+44 47 3516 5895',
	'address': {
		'address_line_1': '510 Henalta',
		'address_line_2': None,
		'city': 'Lyme Regis',
		'country': 'Northern Ireland',
		'post_code': 'TO6Q 8CM',
		'coordinates': [-34.345071, 118.564172]
		}
	};

-- relate into the order table"""+
f"RELATE {random_person_id} -> order:uuid() -> {random_product_id}"+"""
CONTENT {
        "currency": "£",
        "discount": ->product.discount,
        "order_date": time::now(),
        "order_status": "pending",
        "payment_method": "PayPal",
        "price": ->product.price,
        "product_name": ->product.name,
        "quantity": 1,
        "shipping_address": <-person.address
	};

-- update the product table to reduce the quantity"""+
f"""
UPDATE {random_product_id} SET quantity -= 1 RETURN NONE;
COMMIT TRANSACTION;
""")


852 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


 ### Q10: "Transaction"* - New Artist creates their first product

In [21]:
%%timeit
# Transaction - New Artist creates their first product

new_artist_id = str(uuid4())
new_product_id = str(uuid4())

db.query(""" 
BEGIN TRANSACTION;
-- insert into the artist table
CREATE artist CONTENT {"""+
        f"'id': 'artist:⟨{new_artist_id}⟩',"+"""
        'first_name': 'Anderson',
        'last_name': 'West',
        'name': 'Anderson West',
        'company_name': 'Atkins(ws) (ATK)',
        'email': 'six1933@gmail.com',
        'phone': '056 5881 1126',
        'address': {
                'address_line_1': '639 Connaugh',
                'address_line_2': None,
                'city': 'Ripon',
                'country': 'Scotland',
                'post_code': 'CG3U 4TH',
                'coordinates': [4.273648, -112.907273]
                }
        };

-- insert into the product table
CREATE product CONTENT {"""+
        f"'id': 'product:⟨{new_product_id}⟩',"+"""
        'name': 'managed edt allocated pda',
        'description': 'counseling dildo greek pan works interest xhtml wrong dennis available cl specific next tower webcam peace magic',
        'category': 'watercolor',
        'price': 15735.96,
        'currency': '£',
        'discount': None,
        'quantity': 1,
        'image_url': 'https://source.unsplash.com/1920x1080?'
        };

-- relate into the create table"""+
f"RELATE artist:⟨{new_artist_id}⟩ -> create:uuid() -> product:⟨{new_product_id}⟩"+"""
CONTENT {
        created_at: time::now(),
        quantity: 1
};
COMMIT TRANSACTION;
""")

991 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
